In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
# Create a local StreamingContext with two working thread and batch interval of 10 seconds
sc = SparkContext("local[2]", "Contarpalabras")
ssc = StreamingContext(sc, 2)

### Comience a transmitir los datos
Para recibir datos, el contexto de transmisión proporciona un método para transmitir datos desde una conexión de socket TCP o desde archivos como orígenes de entrada. Las fuentes pueden ser fuentes como HDFS, S3, etc. Para leer archivos de texto, existe el método textFileStream de javastreamingcontext.

Pero no podrá leer los archivos ya presentes en el directorio antes de que comience el contexto de transmisión, ya que solo lee los archivos recién creados.

Así que aquí transmitiré los datos a través de la conexión de socket a través del puerto 9999 y crearé un receptor java de entrada DStream con él.

In [ ]:
lines = ssc.socketTextStream('localhost', 9999)
ssc.checkpoint("file:///spark")

Así que ahora, si establece una conexión de socket y escribe algo en el terminal, y ejecuta el dstream, verá que el texto aparece en la consola.

Nota: Para iniciar un contexto de transmisión java, debemos decirle a spark que lo inicie, esperar a que finalice el cálculo y luego detenerlo. Y necesitamos imprimir el DStream por el método pprint().

Ejecute en el cmd python server_console.py

In [ ]:
counts = lines.flatMap(lambda line: line.split(","))\
    .filter(lambda word:"ERROR" in word)\
    .map(lambda word : (word, 1))\
    .reduceByKey(lambda a, b : a + b)
counts.pprint()
ssc.start()
ssc.awaitTermination()

In [ ]:
 ## window size = 10, sliding interval = 2
# Checkpoint for backups
ssc.checkpoint("file:///spark")
counts = lines.countByWindow(10, 2)
## Display the counts
## Start the program
## The program will run until manual termination
counts.pprint()
ssc.start()
ssc.awaitTermination()

### reduceByWindow
Transformación con estado mediante Dstreams

In [ ]:
## summary function
## reverse function
## window size = 10
## sliding interval = 2
sum = lines.reduceByWindow(
    lambda x, y: int(x) + int(y),
    lambda x, y: int(x) - int(y),
    10, 2)

## Display the counts
## Start the program
## The program will run until manual termination
sum.pprint()
ssc.start()
ssc.awaitTermination()